# Linear regression

In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lr").getOrCreate()

22/11/30 21:13:26 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.15.11 instead (on interface wlp3s0)
22/11/30 21:13:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/30 21:13:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/30 21:13:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format("libsvm").load("../../data/sample_linear_regression_data.txt")

22/11/30 21:13:34 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
training.show(2)

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
|-9.490009878824548|(10,[0,1,2,3,4,5,...|
|0.2577820163584905|(10,[0,1,2,3,4,5,...|
+------------------+--------------------+
only showing top 2 rows



In [8]:
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    predictionCol="prediction"
).fit(
    training
)

22/11/30 21:14:27 WARN Instrumentation: [1a24ea7a] regParam is zero, which might cause numerical instability and overfitting.


In [11]:
lr.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
lr.intercept

0.14228558260358093

In [14]:
lr_summary = lr.summary

In [15]:
lr_summary.r2

0.027839179518600154

## Train-test split

In [16]:
data = spark.read.format("libsvm").load("../../data/sample_linear_regression_data.txt")

22/11/30 21:16:26 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [17]:
train, test = data.randomSplit([0.7, 0.3])

In [18]:
train.show(2)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 2 rows



In [21]:
test.show(2)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 2 rows



In [22]:
print("train size ", train.count())
print("test size ", test.count())

train size  349
test size  152


In [23]:
new_lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    predictionCol="prediction"
).fit(train)

22/11/30 21:18:59 WARN Instrumentation: [637eb63d] regParam is zero, which might cause numerical instability and overfitting.


In [24]:
test_results = new_lr.evaluate(test)

In [26]:
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-26.974099000161683|
| -22.24469196684521|
| -22.07234509454316|
|  -26.1994993327287|
|-23.614320951669196|
+-------------------+
only showing top 5 rows



In [35]:
new_lr.transform(test.select("features")).show(5)

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -1.071919037614951|
|(10,[0,1,2,3,4,5,...|-1.2661921234777624|
|(10,[0,1,2,3,4,5,...| -1.415095026393351|
|(10,[0,1,2,3,4,5,...| 3.2496733965326237|
|(10,[0,1,2,3,4,5,...|  3.402243692710523|
+--------------------+-------------------+
only showing top 5 rows

